In [1]:
import torch
import numpy as np

In [2]:
mat = np.load("viper//viper_features.npy", allow_pickle=True)

In [3]:
EPS = 0.001 # the regularization constant
sigma = 2**-16 # the kernel width
N = 632 # number of persons
d = 100 # number of features

### Loading parameters

In [4]:
idxa = mat.item().get('idxa')-1
idxa = idxa.astype('float64')
idxa = torch.from_numpy(idxa)

idxb = mat.item().get('idxb')-1
idxb = idxb.astype('float64')
idxb = torch.from_numpy(idxb)

X = mat.item().get('ux')[:d, :] 
X = torch.from_numpy(X)

print(idxa.shape, idxb.shape, X.shape)

torch.Size([1, 632]) torch.Size([1, 632]) torch.Size([100, 1264])


### Data Review

In [5]:
X[:,0]

tensor([ 5.6763e+01, -2.1639e+01, -2.6869e+01, -2.6857e+01, -1.2353e+01,
         9.6739e+00, -2.4680e+00, -4.1892e+01, -6.1503e+00, -1.8926e+01,
        -8.1701e-01,  3.4005e+01,  1.1992e+01, -3.7692e+01,  1.2938e+01,
         1.9612e+01, -4.2859e-01,  2.0604e+01, -4.4400e+00,  8.6628e+00,
        -1.4934e+01, -1.7920e+01, -2.0539e+00,  5.2963e+00, -1.0537e+01,
        -1.1793e+01, -1.3612e+01, -4.4364e+00, -3.4284e+00, -3.7028e+00,
        -9.3622e+00,  6.6155e+00,  1.9674e+01, -9.0171e-01, -8.4710e+00,
         9.8897e-01, -7.4855e+00,  1.9710e+01, -5.8935e+00,  6.1913e+00,
         2.1174e+00, -8.7990e+00,  5.6792e+00, -1.3161e+01,  2.8724e-01,
        -5.4849e+00, -4.0117e+00,  6.0003e+00, -8.6844e+00,  2.5930e+00,
        -5.0884e+00,  4.9267e+00, -2.0084e+01,  1.0568e+01, -1.3966e+01,
         5.3753e+00, -6.1388e+00,  5.9289e+00, -4.0125e+00,  1.8924e+00,
        -3.0191e+00,  5.9632e+00, -2.4214e+00,  9.8576e-01, -2.9458e+00,
         3.2906e+00, -3.8012e+00,  1.4486e+01, -3.5

In [6]:
perm = torch.randperm(N)

In [7]:
idxtrain = perm[:N//2]
idxtest = perm[N//2:]

In [8]:
idxtrain.shape

torch.Size([316])

In [9]:
first_ind = torch.cat((idxa[0,idxtrain], idxa[0, idxtrain]))
first_ind = first_ind.view(1, first_ind.shape[0])
second_ind = torch.cat((idxb[0, idxtrain], idxb[0, idxtrain[torch.randperm(N//2)]] ))
second_ind = second_ind.view(1, second_ind.shape[0])

In [10]:
print(first_ind.shape, second_ind.shape)

torch.Size([1, 632]) torch.Size([1, 632])


In [11]:
temp_a = torch.ones(N)
temp_b = torch.cat((torch.ones(N//2), torch.zeros(N//2)))
matches = torch.logical_and(temp_a, temp_b)

In [12]:
S = torch.cat((first_ind[:, matches], second_ind[:, matches]))
D = torch.cat((first_ind[:, torch.logical_not(matches)], second_ind[:, torch.logical_not(matches)]))

### Compute kernelmatrix

In [13]:
from kernelmatrix import kernelmatrix
K = kernelmatrix(X=X, X2=torch.tensor([]), sigma=sigma)

### Compute H

In [14]:
n = K.shape[0]
B = torch.zeros(n, 1)
E = torch.zeros(n, 1)
W = torch.zeros(n, n)

In [15]:
ib = D[0, :].type(torch.int64)
# .view(1, D.shape[1])
ie = D[1, :].type(torch.int64)
# .view(1, D.shape[1])
print(ib.shape, ie.shape)

torch.Size([316]) torch.Size([316])


In [16]:
t = torch.ones(316,1)
print(t.shape)
# print(t)

torch.Size([316, 1])


In [17]:
B[ib.type(torch.int64)] = t

In [18]:
E[ie.type(torch.int64)] = t

In [19]:
for i in range(0, ib.shape[0]):
    W[ib[i], ie[i]] = W[ib[i],ie[i]]+1

In [20]:
from computeH import computeH
H = computeH(1264, ib=D[0,:], ie=D[1,:])

In [21]:
torch.count_nonzero(torch.diagonal(H))
# matlab output nnz(diag(H0)): 632
# NO DOUBT we're calculating H's correctly.

tensor(632)

### kernel function


In [22]:
n = K.shape[0]
e = EPS
n1 = S.shape[1]
n0 = D.shape[1]
H0 = computeH(n, D[0, :], D[1, :])
H1 = computeH(n, S[0, :], S[1, :])

In [23]:
from helps.kernel import computeT
C = computeT(n0, 0.001, H0, K)-computeT(n1, 0.001, H1, K)

In [24]:
torch.count_nonzero(C)

tensor(399424)

In [25]:
torch.diagonal(C)[:100]

tensor([-246.5799,    0.0000, -705.9613,    0.0000,    0.0000,    0.0000,
         -83.9442, -449.1142, -269.7489,   21.9998,    0.0000,    0.0000,
        -116.6399,    0.0000,  -11.6200,  -84.0050,    0.0000,    0.0000,
          -1.7168,    0.0000,    0.0000,  326.0536, -531.5749,    0.0000,
           0.0000,    0.0000, -226.2907,  -43.9010,    0.0000,  133.7894,
           0.0000,    0.0000,    0.0000,    0.0000, -288.2025,    0.0000,
           0.0000,  170.8136, -317.2287,   35.5850, -341.0009,  -49.9253,
        -170.3652,    0.0000,   98.0961,    0.0000,   20.9441,  432.5608,
           0.0000,    0.0000,    0.0000,    0.0000,  -60.5735,    0.0000,
           0.0000,    0.0000, -688.4719,   38.0362, -329.2959,   31.8932,
        -242.3010,  505.2974, -422.7775,    0.0000,   -8.8691,    0.0000,
           0.0000,   92.2994,    0.0000,    0.0000, -302.1781,    0.0000,
        -329.5821,  222.2434,    0.0000,  135.9778,    0.0000,    0.0000,
        -130.1544,  220.8037,  -33.029

In [26]:
# from kernelmatrix import kernelmatrix

# K  = kernelmatrix(X=X, X2=torch.tensor([]), sigma=sigma);       

# part_A = (1/(n*EPS**2))
# print(part_A)

# t = torch.eye(K.shape[0]) + (1/(n*2))*torch.matmul(K.double(), H.double())
# torch.count_nonzero(t)

# part_B = torch.inverse(t)
# print(torch.count_nonzero(part_B))

# C = part_A * torch.matmul(H.double(), part_B)
# torch.count_nonzero(C)

In [27]:
from helps.kernel import kernel
M = kernel(EPS, S, D, K, pmetric=1)
print(M.shape)
# print(torch.diagonal(M)[:100])
# print(M[:100,:10])

torch.Size([1264, 1264])


### project to positive-semidefinite cone

In [28]:
# a = torch.tensor([[0.6529, 0.1343, 0.1778], [0.4367, 0.5539, 0.9764], [0.5284, 0.6233, 0.3749]])
# print(a.shape)
# print(a)
# x, y = torch.eig(a)
# print(x)
# print(y)

In [59]:
D, V = torch.eig(M, eigenvectors=True)
d = D[:,0]
for idx, i in enumerate(d):
    if i<=0:
        d[idx] = torch.finfo(torch.float64).eps

In [71]:
D_ = torch.diag(d).type(torch.double)
V = V.type(torch.double)
M = torch.mm(V, torch.mm(D_, V.T))
# torch.diagonal(M)[1:100]